# Code to predict the output

mount the drive and unzip archive to retrive test folder


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!unzip '/content/drive/My Drive/Assignment.zip'

Archive:  /content/drive/My Drive/Assignment.zip
replace Assignment/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
from keras import backend as K
from keras import optimizers
import h5py
import pandas as pd

In [0]:
# image folder
folder_path = '/content/Assignment/Input/Dataset/test'
# path to model
model_path = '/content/drive/My Drive/category_model.h5'
# dimensions of images for model trained, please replace with new model as you see fit
img_width, img_height = 150, 150

# Helper Function
define the helper functions that were used to assess model

In [0]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
def mcor(y_true, y_pred):
     #matthews_correlation
     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
     y_pred_neg = 1 - y_pred_pos
 
 
     y_pos = K.round(K.clip(y_true, 0, 1))
     y_neg = 1 - y_pos
 
 
     tp = K.sum(y_pos * y_pred_pos)
     tn = K.sum(y_neg * y_pred_neg)
 
 
     fp = K.sum(y_neg * y_pred_pos)
     fn = K.sum(y_pos * y_pred_neg)
 
 
     numerator = (tp * tn - fp * fn)
     denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
     return numerator / (denominator + K.epsilon())

In [0]:
# load the trained model
model = load_model(model_path,custom_objects={"f1": f1,"mcor":mcor})
model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.Adamax(lr=1e-4),
          metrics=['accuracy',f1,mcor])


In [0]:
# load all images into a list
images = []
for img in os.listdir(folder_path):
  
  if img==".DS_Store":
    continue
  img = image.load_img(folder_path+"/"+img, target_size=(img_width, img_height),grayscale=True)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  images.append(img)




/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [0]:
# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)

In [0]:
## print classes
'''
0: category 1
1: category 2
2: category 3
3: category 4
results are stored in drive in results.csv along with labels
predictions are directly converted to csv after appropriate and convenient
naming
'''
i=0
for img in os.listdir(folder_path):
  
  if img==".DS_Store":
    continue
  print(img," , ",classes[i])
  i=i+1

Test_original_6053.png_62171105-bdaa-4eb5-b5e7-9b0fb7914715.png  ,  1
Test_original_6053.png_c34c8352-0830-4853-81ce-3ea5ecf82313.png  ,  1
Test_original_1052.png_22a82512-5b1f-4072-a4af-9a8af40653f0.png  ,  0
6052.png  ,  1
C053.png  ,  3
6053.png  ,  1
Test_original_6053.png_16f2063e-9590-4ae5-b7f9-7f23c8cfa654.png  ,  1
Test_original_C053.png_e7d20702-793e-4207-b174-a8042c05ac3d.png  ,  3
Test_original_4052.png_31cd9a66-9edc-4445-b4bc-9b0239b342e0.png  ,  2
4043.png  ,  2
Test_original_6052.png_51c87ecf-ede6-4152-960f-4d56ff913d53.png  ,  1
1051.png  ,  0
Test_original_1051.png_271f7fb1-f0ab-4ca7-9dae-a352fd0ed8b9.png  ,  0
4052.png  ,  2
4050.png  ,  2
Test_original_C053.png_84d8e7c9-490f-4579-b9a9-501a8ca02cfe.png  ,  3
Test_original_4043.png_fffcbc29-3aff-40c0-bdf9-2c12fa6df60b.png  ,  2
Test_original_1052.png_b95bc311-bae0-48a7-8b9f-d0eddf1ddf2b.png  ,  0
4051.png  ,  2
Test_original_6052.png_9950e35a-38c7-4388-b391-ce50bb8ea1a7.png  ,  1
Test_original_4052.png_78b7cca7-2a16-4f3